In [1]:
from functools import cache

GOALS = {"A": 0, "B": 1, "C": 2, "D": 3}
COSTS = {"A": 1, "B": 10, "C": 100, "D": 1000}
ROOM_SPOTS = (2, 4, 6, 8)
HALLWAY_SPOTS = (0, 1, 3, 5, 7, 9, 10)
HALLWAY_START = tuple(None for _ in range(len(ROOM_SPOTS) + len(HALLWAY_SPOTS)))

@cache
def solve(rooms, hallway=None):
    hallway = hallway or HALLWAY_START
    if rooms == (("A",) * 4, ("B",) * 4, ("C",) * 4, ("D",) * 4):
        return 0

    best_cost = float('inf')
    for i, piece in enumerate(hallway):
        if piece is None:
            continue
        goal = GOALS[piece]
        can_move = True
        for neighbour in rooms[goal]:
            if neighbour is not None and neighbour != piece:
                can_move = False
                break
        if not can_move:
            continue
        offset = 1 if ROOM_SPOTS[goal] > i else -1
        for j in range(i + offset, ROOM_SPOTS[goal] + offset, offset):
            if hallway[j] is not None:
                can_move = False
                break
        if not can_move:
            continue
        none_count = sum(elem is None for elem in rooms[goal])
        new_room = (None,) * (none_count - 1) + (piece,) * (4 - none_count + 1)
        steps = none_count + abs(i - ROOM_SPOTS[goal])
        cost = steps * COSTS[piece]
        new_rooms = rooms[:goal] + (new_room,) + rooms[goal + 1:]
        new_hallway = hallway[:i] + (None,) + hallway[i + 1:]
        new_cost = cost + solve(new_rooms, new_hallway)
        if new_cost < best_cost:
            best_cost = new_cost
    for i, room in enumerate(rooms):
        wants_to_move = False
        for item in room:
            if item is not None and GOALS[item] != i:
                wants_to_move = True
        if not wants_to_move:
            continue
        none_count = sum(elem is None for elem in room)
        steps = none_count + 1
        piece = room[none_count]
        for hall_goal in HALLWAY_SPOTS:
            destination_steps = steps + abs(hall_goal - ROOM_SPOTS[i])
            destination_cost = destination_steps * COSTS[piece]
            blocked = False
            for j in range(min(hall_goal, ROOM_SPOTS[i]), max(hall_goal, ROOM_SPOTS[i]) + 1):
                if hallway[j] is not None:
                    blocked = True
                    break
            if blocked:
                continue
            new_room = (None,) * (none_count + 1) + room[none_count + 1:]
            new_rooms = rooms[:i] + (new_room,) + rooms[i + 1:]
            new_hallway = hallway[:hall_goal] + (piece,) + hallway[hall_goal + 1:]
            new_cost = destination_cost + solve(new_rooms, new_hallway)
            if new_cost < best_cost:
                best_cost = new_cost
    return best_cost

In [2]:
%%time
solve((('D', 'B'), ('D', 'A'), ('C', 'B'), ('C', 'A')))

CPU times: user 1.29 s, sys: 18.3 ms, total: 1.3 s
Wall time: 1.3 s


16059

In [3]:
%%time
solve((('D', 'D', 'D', 'B'), ('D', 'C', 'B', 'A'), ('C', 'B', 'A', 'B'), ('C', 'A', 'C', 'A')))

CPU times: user 1.9 s, sys: 12.8 ms, total: 1.92 s
Wall time: 1.92 s


43117

In [4]:
solve.cache_info()

CacheInfo(hits=211469, misses=163815, maxsize=None, currsize=163815)